In [32]:
import requests

def query_api(company, prompt):
    url = "https://api.vectara.io/v1/query"
    payload = {
        "query": [
            {
                "query": prompt,
                "queryContext": "",
                "start": 0,
                "numResults": 1,
                "contextConfig": {
                    "charsBefore": 0,
                    "charsAfter": 0,
                    "sentencesBefore": 2,
                    "sentencesAfter": 2,
                    "startTag": "%START_SNIPPET%",
                    "endTag": "%END_SNIPPET%",
                },
                "corpusKey": [
                    {
                        "customerId": "43437896",
                        "corpusId": 2,
                        "semantics": 0,
                        # "metadataFilter": f"doc.Company='{company}'",
                        "lexicalInterpolationConfig": {"lambda": 1},
                        "dim": [],
                    }
                ],
                "summary": [
                    {
                        "debug": False,
                        "chat": {"store": True, "conversationId": ""},
                        "maxSummarizedResults": 3,
                        "responseLang": "eng",
                        "summarizerPromptName": "vectara-summary-ext-v1.2.0",
                        "factualConsistencyScore": True,
                    }
                ],
            }
        ]
    }

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "x-api-key": "zwt_ApbPSAKP2pwNkh7GwzOg7dNP2f94ng3G9_3FLA",
        "customer-id": "43437896",
    }
    
    
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            return response.json()
        else:
            print("Error:", response.text)
            return None
    except Exception as e:
        print("Error:", str(e))
        return None


In [33]:
# Example usage:
company_name = "gitlab"
prompt_name = "how to create an account"
result = query_api(company_name, prompt_name)

In [34]:
result

{'responseSet': [{'response': [{'text': "Mark Pincus has kept control of Zynga's too. But are these just outliers? %START_SNIPPET%How common is it for founders to keep control after an A round?%END_SNIPPET% I'd heard of several cases among the companies we've funded, but I wasn't sure how many there were, so I emailed the ycfounders list. The replies surprised me.",
     'score': 11.926043,
     'metadata': [{'name': 'lang', 'value': 'eng'},
      {'name': 'section', 'value': '1'},
      {'name': 'offset', 'value': '794'},
      {'name': 'len', 'value': '63'}],
     'documentIndex': 0,
     'corpusKey': {'customerId': 0,
      'corpusId': 2,
      'semantics': 'DEFAULT',
      'dim': [],
      'metadataFilter': '',
      'lexicalInterpolationConfig': None},
     'resultOffset': 89,
     'resultLength': 63}],
   'status': [],
   'document': [{'id': 'control.txt',
     'metadata': [{'name': 'X-TIKA:Parsed-By',
       'value': 'org.apache.tika.parser.csv.TextAndCSVParser'},
      {'name':

In [ ]:

api_key_header = {
    "customer-id": 43437896,
    "x-api-key": "zwt_ApbPSAKP2pwNkh7GwzOg7dNP2f94ng3G9_3FLA"
}
 
data_dict = {
    "query": [
        {
            "query": "What is the meaning of life?",
            "num_results": 1,
            "corpus_key": [
                {
                    "customer_id": api_key_header["customer-id"],
                    "corpus_id": api_key_header["x-api-key"]
                }
            ]
        }
    ]
}
payload = json.dumps(data_dict)
response = requests.post(
    "https://api.vectara.io/v1/query",
    data=payload,
    verify=True,
    headers=api_key_header)

In [35]:
import argparse
import json
import logging
import sys

import requests

def _get_query_json(customer_id: int, corpus_id: int, query_value: str):
    """Returns a query JSON."""
    query = {
        "query": [
            {
                "query": query_value,
                "num_results": 10,
                "corpus_key": [{"customer_id": customer_id, "corpus_id": corpus_id}],
            },
        ],
    }
    return json.dumps(query)


In [36]:
def query(customer_id: int, corpus_id: int, query_address: str, api_key: str, query: str):
    """Queries the data.

    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        query_address: Address of the querying server. e.g., api.vectara.io
        api_key: A valid API key with query access on the corpus.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """
    post_headers = {
        "customer-id": f"{customer_id}",
        "x-api-key": api_key
    }

    response = requests.post(
        f"https://{query_address}/v1/query",
        data=_get_query_json(customer_id, corpus_id, query),
        verify=True,
        headers=post_headers)

    if response.status_code != 200:
        logging.error("Query failed with code %d, reason %s, text %s",
                       response.status_code,
                       response.reason,
                       response.text)
        return response, False

    message = response.json()
    if (message["status"] and
        any(status["code"] != "OK" for status in message["status"])):
        logging.error("Query failed with status: %s", message["status"])
        return message["status"], False

    for response_set in message["responseSet"]:
        for status in response_set["status"]:
            if status["code"] != "OK":
                return status, False

    return message, True

In [38]:
logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s", level=logging.INFO)

customer_id = 43437896
corpus_id = 3
serving_endpoint = "api.vectara.io"
api_key = "zwt_ApbPSAKP2pwNkh7GwzOg7dNP2f94ng3G9_3FLA"
query = "Test query"

response, status = query(customer_id,
                         corpus_id,
                         serving_endpoint,
                         api_key,
                         query)

logging.info("Query response: %s", response)

TypeError: 'str' object is not callable